# 1.import data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('E:\data-science\Techlent\Project\data.csv')

In [3]:
df.head()

,address,rent,property_specification,available_date,agent_name,agent_company_name,property_feature,property_description,domain_say,school_catchment_zone,neighbourhood_insights,market_performance,demographics
0,"201/8-10 New McLean Street, Edgecliff NSW 2027",$650 Per Week,− 0 Beds1 Bath1 Parking,Available Now,Lana Gojak,Highland Double Bay,Property FeaturesClose to shopsClose to transport,Property DescriptionSPACIOUS STUDIOQuietly nes...,"201/8-10 New McLean Street, Edgecliff NSW 2027...",Government School CatchmentWoollahra Public Sc...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: -Auction clear...,"DemographicsPopulation: 2,576Occupancy: Owner:..."
1,"1354/37 King Street, Sydney NSW 2000",$775 | FURNISHED,1 Bed1 Bath− Parking,Available Now,Martin Chuah,Century 21 City Quarter,Property FeaturesFurnishedNo pet allowedAir co...,Property DescriptionOPEN INSPECTIONS NOW AVAIL...,"First listed on 11 January, this unit has been...",Government School CatchmentFort Street Public ...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $810kAuction c...,"DemographicsPopulation: 17,241Occupancy: Owner..."
2,"3/201 Darlinghurst Road, Darlinghurst NSW 2010",$730 per week,1 Bed1 Bath1 Parking,Available Now,Sonia Beresford,BresicWhitney Inner East,property_feature not found,Property DescriptionAvailable fully furnished ...,"First listed on 10 January, this unit has been...",Government School CatchmentCrown Street Public...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $720kAuction c...,"DemographicsPopulation: 11,322Occupancy: Owner..."
3,"5 Simeon Street, Clovelly NSW 2031","$1,950 pw",4 Beds2 Baths− Parking,Available Now,Mimi Charles,McGrath Coogee,Property FeaturesArea ViewsPets AllowedClose t...,"Property DescriptionCreative, Soulful Family H...","First listed on 18 December, this house has be...",Government School CatchmentClovelly Public Sch...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $5.2mAuction c...,"DemographicsPopulation: 4,752Occupancy: Owner:..."
4,"2/18 Thornton Street, Darling Point NSW 2027","$4,495",4 Beds3 Baths2 Parking,Available Now,agent_name not found,Jeffrey Lamens,Property FeaturesEnsuite(s)Secure ParkingAir c...,Property DescriptionLuxurious Private House-Si...,"2/18 Thornton Street, Darling Point NSW 2027 w...",Government School CatchmentDouble Bay Public S...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: -Auction clear...,"DemographicsPopulation: 4,192Occupancy: Owner:..."


In [4]:
df.columns

Index(['address', 'rent', 'property_specification', 'available_date',
       'agent_name', 'agent_company_name', 'property_feature',
       'property_description', 'domain_say', 'school_catchment_zone',
       'neighbourhood_insights', 'market_performance', 'demographics'],
      dtype='object')

In [5]:
df.index

RangeIndex(start=0, stop=839, step=1)

In [6]:
df.shape

(839, 13)

# 2. EDA for each columns

In [7]:
!pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.3 kB ? eta -:--:--
     -------------------------------------- 40.3/40.3 kB 485.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 991.0 kB/s eta 0:00:01
   ---------------------- ----------------- 71.7/125.4 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 71.7/125.4 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 738.4 kB/s eta 0:00:00


In [8]:
from geopy.geocoders import Nominatim
import re

def preprocess_address(address):
    # 拆分地址
    address_parts = re.split(r'[, ]+', address)
    
    # 初始化部分变量
    street_number = ''
    street_name = ''
    city = ''
    state = ''
    postal_code = ''
    
    # 遍历地址部分并分配到相应的变量
    for part in address_parts:
        if part.isdigit():
            street_number = part
        elif part.isalpha():
            street_name += part + ' '
        elif part.isnumeric() and len(part) == 4:
            postal_code = part
        else:
            # 假设剩余的部分为城市和州
            if not state:
                state = part
            elif not city:
                city = part
    
    # 清理和规范化
    street_name = street_name.strip().title()
    city = city.title()
    state = state.upper()
    
    return street_number, street_name, city, state, postal_code

def geocode_address(address):
    # 使用Nominatim进行地理编码
    geolocator = Nominatim(user_agent="address_geocoder")
    location = geolocator.geocode(address)
    
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None, None

# 示例地址
example_address = "201/8-10 New McLean Street, Edgecliff NSW 2027"

# 预处理地址
street_number, street_name, city, state, postal_code = preprocess_address(example_address)
print("Street Number:", street_number)
print("Street Name:", street_name)
print("City:", city)
print("State:", state)
print("Postal Code:", postal_code)

# 地理编码
latitude, longitude = geocode_address(example_address)
print("Latitude:", latitude)
print("Longitude:", longitude)


Street Number: 2027
Street Name: New Mclean Street Edgecliff Nsw
City: 
State: 201/8-10
Postal Code: 
Latitude: None
Longitude: None


In [9]:
import pandas as pd
import re
from geopy.geocoders import Nominatim


def preprocess_address(address):
    address_parts = re.split(r',', address)

    # 命名分割后的两个部分
    if len(address_parts) == 2:
        address_part1 = address_parts[0].strip()
        address_part2 = address_parts[1].strip()
    else:
        print("无法根据逗号成功分割地址")
    
    # 分别根据空格分隔
    adress_part1_components = re.split(r'\s+', adress_part1)
    address_part2_components = re.split(r'\s+', address_part2)
    
    adress_part1_components_numbers = address_part1_components[0].strip()
    
    adress_part1_components_numbers_number = re.split(r'\\', adress_part1_components_numbers)

    

    street_number = adress_part1_components_numbers_number[0].strip()
    unit_numbers = adress_part1_components_numbers_number[1].strip()
    street_name = address_part1_components[1].strip()
    suburb = address_part2_components[0].strip()
    state = address_part2_components[1].strip()
    postal_code = address_part2_components[2].strip()
    
    
    return street_number,unit_numbers,street_name, suburb, state, postal_code

# 对 DataFrame 中的地址列应用预处理函数，将结果存储在新列中
df[['street_number', 'street_name', 'city', 'state', 'postal_code']] = df['address'].apply(preprocess_address).apply(pd.Series)

# 输出处理后的 DataFrame
print(df)


                                             address  \
0     201/8-10 New McLean Street, Edgecliff NSW 2027   
1               1354/37 King Street, Sydney NSW 2000   
2     3/201 Darlinghurst Road, Darlinghurst NSW 2010   
3                 5 Simeon Street, Clovelly NSW 2031   
4       2/18 Thornton Street, Darling Point NSW 2027   
..                                               ...   
834  3.05/300 Oxford Street, Bondi Junction NSW 2022   
835          29 Jennings Street, Matraville NSW 2036   
836  21/1 Bennett Avenue, Strathfield South NSW 2136   
837        601/5-7 Albert Road, Strathfield NSW 2135   
838     1401/197 Castlereagh Street, Sydney NSW 2000   

                              rent   property_specification  \
0                    $650 Per Week  − 0 Beds1 Bath1 Parking   
1                 $775 | FURNISHED     1 Bed1 Bath− Parking   
2                    $730 per week     1 Bed1 Bath1 Parking   
3                        $1,950 pw   4 Beds2 Baths− Parking   
4           

In [10]:
df.head()

,address,rent,property_specification,available_date,agent_name,agent_company_name,property_feature,property_description,domain_say,school_catchment_zone,neighbourhood_insights,market_performance,demographics,street_number,street_name,city,state,postal_code
0,"201/8-10 New McLean Street, Edgecliff NSW 2027",$650 Per Week,− 0 Beds1 Bath1 Parking,Available Now,Lana Gojak,Highland Double Bay,Property FeaturesClose to shopsClose to transport,Property DescriptionSPACIOUS STUDIOQuietly nes...,"201/8-10 New McLean Street, Edgecliff NSW 2027...",Government School CatchmentWoollahra Public Sc...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: -Auction clear...,"DemographicsPopulation: 2,576Occupancy: Owner:...",2027,New Mclean Street Edgecliff Nsw,,201/8-10,
1,"1354/37 King Street, Sydney NSW 2000",$775 | FURNISHED,1 Bed1 Bath− Parking,Available Now,Martin Chuah,Century 21 City Quarter,Property FeaturesFurnishedNo pet allowedAir co...,Property DescriptionOPEN INSPECTIONS NOW AVAIL...,"First listed on 11 January, this unit has been...",Government School CatchmentFort Street Public ...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $810kAuction c...,"DemographicsPopulation: 17,241Occupancy: Owner...",2000,King Street Sydney Nsw,,1354/37,
2,"3/201 Darlinghurst Road, Darlinghurst NSW 2010",$730 per week,1 Bed1 Bath1 Parking,Available Now,Sonia Beresford,BresicWhitney Inner East,property_feature not found,Property DescriptionAvailable fully furnished ...,"First listed on 10 January, this unit has been...",Government School CatchmentCrown Street Public...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $720kAuction c...,"DemographicsPopulation: 11,322Occupancy: Owner...",2010,Darlinghurst Road Darlinghurst Nsw,,3/201,
3,"5 Simeon Street, Clovelly NSW 2031","$1,950 pw",4 Beds2 Baths− Parking,Available Now,Mimi Charles,McGrath Coogee,Property FeaturesArea ViewsPets AllowedClose t...,"Property DescriptionCreative, Soulful Family H...","First listed on 18 December, this house has be...",Government School CatchmentClovelly Public Sch...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: $5.2mAuction c...,"DemographicsPopulation: 4,752Occupancy: Owner:...",2031,Simeon Street Clovelly Nsw,,,
4,"2/18 Thornton Street, Darling Point NSW 2027","$4,495",4 Beds3 Baths2 Parking,Available Now,agent_name not found,Jeffrey Lamens,Property FeaturesEnsuite(s)Secure ParkingAir c...,Property DescriptionLuxurious Private House-Si...,"2/18 Thornton Street, Darling Point NSW 2027 w...",Government School CatchmentDouble Bay Public S...,Neighbourhood InsightsA little bit about who l...,Market performanceMedian price: -Auction clear...,"DemographicsPopulation: 4,192Occupancy: Owner:...",2027,Thornton Street Darling Point Nsw,,2/18,


In [13]:
import pandas as pd
import re
from geopy.geocoders import Nominatim


def preprocess_address(address):
    address_parts = re.split(r',', address)

    # 命名分割后的两个部分
    if len(address_parts) == 2:
        address_part1 = address_parts[0].strip()
        address_part2 = address_parts[1].strip()
    else:
        print("无法根据逗号成功分割地址")
    
    # 分别根据空格分隔
    adress_part1_components = re.split(r'\s+', adress_part1)
    address_part2_components = re.split(r'\s+', address_part2)
    
    adress_part1_components_numbers = address_part1_components[0].strip()
    
    adress_part1_components_numbers_number = re.split(r'/', adress_part1_components_numbers)

    

    street_number = adress_part1_components_numbers_number[0].strip()
    unit_numbers = adress_part1_components_numbers_number[1].strip()
    street_name = address_part1_components[1].strip()
    suburb = address_part2_components[0].strip()
    state = address_part2_components[1].strip()
    postal_code = address_part2_components[2].strip()
    
    
    return street_number,unit_numbers,street_name, suburb, state, postal_code

In [14]:
preprocess_address(201/8-10 New McLean Street, Edgecliff NSW 2027)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (535449379.py, line 1)

In [ ]:
adress: 201/8-10 New McLean Street, Edgecliff NSW 2027
分类的目标是：street_number = 201
    unit_numbers = 8-10
    street_name = New McLean Street
    suburb =  Edgecliff
    state = NSW
    postal_code=2027
    
    请给出相应的代码

In [26]:
import re


# 定义正则表达式来提取各部分  
street_number = r'(\d+/\d+)'
unit_number = r'\d+'
street_name = r'[A-Za-z\s]+(?:Street|Road|Avenue|Lane|Crescent|Circuit|Place|Close|Drive|Grove|Parade|Walk|Way|Boulevard)'
suburb = r'[A-Za-z\s]+[A-Z]{3}'
state = r'[A-Z]{3}'
postal_code = r'\d{4}'

# 提取并生成各分类列
df['street_number'] = df['address'].str.extract(street_number)
df['unit_number'] = df['address'].str.extract(unit_number) 
df['street_name'] = df['address'].str.extract(street_name)
df['suburb'] = df['address'].str.extract(suburb)
df['state'] = df['address'].str.extract(state)
df['postal_code'] = df['address'].str.extract(postal_code)


ValueError: pattern contains no capture groups

In [17]:
import re

address = "1354/37 King Street, Sydney NSW 2000"

# Extract components
street_number = re.search(r'(\d+)', address).group(1) 
unit_numbers = re.search(r'/(\d+)', address).group(1)
street_name = re.search(r'[A-Z].+ Street', address).group()
suburb = re.search(r'\w+[ ]\w+', address).group()
state = re.search(r'\b[A-Z]{3}\b', address).group()  
postal_code = re.search(r'\d{4}', address).group()

# Print results
print("street_number =", street_number)
print("unit_numbers =", unit_numbers)
print("street_name =",street_name)  
print("suburb =", suburb)
print("state =", state)
print("postal_code =", postal_code)

street_number = 1354
unit_numbers = 37
street_name = King Street
suburb = 1354
state = NSW
postal_code = 1354


In [30]:
address = "1354/37 King Street, Sydney NSW 2000"

In [31]:


# 分割地址各部分
address_parts = address.split(', ')

# 分析地址的街道和单元号
street_and_unit = address_parts[0]
unit_numbers = street_and_unit.split('/')
street_number = unit_numbers[1].strip() if len(unit_numbers) > 1 else None
unit_number = unit_numbers[0].strip() if len(unit_numbers) > 1 else None

street_name = address_parts[1].strip()
suburb = address_parts[2].split(' ')[0].strip()
state = address_parts[2].split(' ')[1].strip()
postal_code = address_parts[2].split(' ')[2].strip()


IndexError: list index out of range